In [97]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.exceptions import NotFittedError
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, f1_score, accuracy_score,recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import (
    accuracy_score,
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    f1_score
)

train = pd.read_csv(
    "../Dataset/Preparation_dir/final_data/feature_engineering_data/version2/train.csv"
)
test = pd.read_csv(
    "../Dataset/Preparation_dir/final_data/feature_engineering_data/version2/test.csv"
)

target = "act"

In [98]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import RandomizedSearchCV

selected_features_list = [
    "BandAccX",
    "BandAccY",
    "BandAccZ",
    "speed",
    "rate",
    "rateZone",
    "usage",
    "deviceStatus",
    "Acceleration_X",
    "Acceleration_Y",
    "Acceleration_Z",
    "MagneticField_X",
    "MagneticField_Y",
    "MagneticField_Z",
    "Orientation_X",
    "Orientation_Y",
    "Orientation_Z",
    "AngularVelocity_X",
    "AngularVelocity_Y",
    "AngularVelocity_Z",
    "act"
]

# 只选择selected_features_list中的特征
X_train = train[selected_features_list]
X_test = test[selected_features_list]


In [99]:
X_train.columns

Index(['BandAccX', 'BandAccY', 'BandAccZ', 'speed', 'rate', 'rateZone',
       'usage', 'deviceStatus', 'Acceleration_X', 'Acceleration_Y',
       'Acceleration_Z', 'MagneticField_X', 'MagneticField_Y',
       'MagneticField_Z', 'Orientation_X', 'Orientation_Y', 'Orientation_Z',
       'AngularVelocity_X', 'AngularVelocity_Y', 'AngularVelocity_Z', 'act'],
      dtype='object')

In [100]:
# 获取数据集中所有常数列的列表
constant_columns = [col for col in X_train.columns if X_train[col].nunique() <= 1]

# 删除这些列
X_train = X_train.drop(columns=constant_columns)
X_test = X_test.drop(columns=constant_columns)

# 创建训练集的特征和目标变量
target = 'act'  # 需要替换为你的目标列名
y_train = X_train[target]
X_train = X_train.drop(target, axis=1)

# 创建测试集的特征和目标变量
y_test = X_test[target]
X_test = X_test.drop(target, axis=1)

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 使用随机森林分类器
rf_model = RandomForestClassifier()

# 使用RFE进行特征选择
selector = RFE(rf_model, n_features_to_select=10, step=1)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# 确保特征名称一致
selected_features = selector.get_support(indices=True)
X_train_selected = pd.DataFrame(
    X_train_selected, columns=X_train.columns[selected_features]
)
X_test_selected = pd.DataFrame(
    X_test_selected, columns=X_train.columns[selected_features]
)

# 定义贝叶斯优化的参数空间
param_space = {
    "n_estimators": (100, 200),  # 进一步缩小范围
    "max_features": ["sqrt"],  # 进一步简化选项
    "max_depth": (10, 30),  # 进一步缩小范围
    "min_samples_split": (2, 5),  # 进一步缩小范围
    "min_samples_leaf": (1, 3),  # 进一步缩小范围
}

# 创建 BayesSearchCV 对象
bayes_search = BayesSearchCV(
    estimator=rf_model,
    search_spaces=param_space,
    n_iter=50,  # 保持迭代次数
    cv=5,
    n_jobs=8,
    scoring="r2",
    verbose=1,
    n_points=3,  # 每次评估多个参数组合
    random_state=0,  # 确保结果可重复
)

# 拟合贝叶斯优化
bayes_search.fit(X_train_selected, y_train)
print("Best parameters found: ", bayes_search.best_params_)
best_rf = bayes_search.best_estimator_

# 预测
y_pred = best_rf.predict(X_test_selected)

# 计算评估指标
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

precision=precision_score(y_test, y_pred, average="macro")
print(f"Precision: {precision}")

recall=recall_score(y_test, y_pred, average="macro")
print(f"Recall: {recall}")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse}")

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

f1 = f1_score(y_test, y_pred, average="macro")
print(f"F1 Score: {f1}")

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 2 candidates